# NYC 311 Capstone - Data Cleaning 
## Stephen Behunin | BrainStation | stephenbehuninwork@gmail.com
__________________________________________________________________________________________________________________________________________________________
## Notebook Executive Summary
__________________________________________________________________________________________________________________________________________________________

The goal of this section is cleaning and formatting the raw data provided by the NYC 311 Data Portal for use in analysis and modeling. Areas of primary concern are missing/invalid data, formatting column datatypes, and filtering the data to the scope of the analysis. 

## Background
_________________________________________________________________________________________________________________________________________________________
The cleaning in this notebook is done in two stages, first the dataset is cleaned on complaint relevant data to ensure that each SR is valid and will be stored in a .csv file. 
Next the entire dataset is condensed into a timeseries format and converted into another .csv file.

## Layout
__________________________________________________________________________________________________________________________________________________________
#### Pre-amble
- Importing packages
- Reading in the source data
- Creating a backup copy of the original dataset

#### Part 1 - Dataset Basics
- Collecting basic information about the dataset

#### Part 2 - Columns
- Dropping unwanted columns
- Top Ten Descriptors
- Formatting column datatypes

#### Part 3 - Rows
- Dealing with missing or null data
- Searching for duplicate values
- Resetting the index column

#### Part 4 - Limiting Scope
- Filtering out unclosed complaints
- Restricting the dataset to complaints from 2010 to 2019
- Converting the complaint dataframe to a .csv file

#### Part 5- Creating the Timeseries Dataframe
- Creating the dataset
- Cleaning the new dataset

#### Part 6- Conversion to CSV
- Final dimensions check
- Converting the dataframe to a .csv file

## Pre-amble
__________________________________________________________________________________________________________________


### Importing packages
__________________________

In [1]:
# Importing Pandas and Numpy packages
import pandas as pd
import numpy as np
from datetime import datetime
# Starting notebook timer
e_start = datetime.now()

### Reading in data
_________________________

In [2]:
start = datetime.now()
# Reading in the data
nyc_311 = pd.read_csv("311 Data/311_Data.csv")

# Displaying the dataframe
nyc_311.head(5)
finish = datetime.now()
duration = finish - start
print(duration)

C:\Users\steph\anaconda3\envs\testing_env\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (8,17,20,31,32,33,34,35,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


0:02:35.774966


### Creating a copy of the original data 
________________________________________
This copy of the data is included for the readers convenience to avoid rerunning the notebook if a restoration of the original data is needed.

In [3]:
# Setting the backup copy to the original dataset
nyc_311_original = nyc_311

Code to implement a reset of the data

In [4]:
# This cell will reset the data to its original state
nyc_311 = nyc_311_original

## Part 1- Dataset Basics
_____________________________

### Gathering basic information on the dataset
_______________________________________________

Checking shape of the dataset.

In [5]:
shape = nyc_311.shape
rows = shape[0]
columns = shape[1]
print(f"The NYC 311 dataframe has {rows} rows and {columns} columns.")

The NYC 311 dataframe has 26256834 rows and 41 columns.


Checking column names and datatypes

In [6]:
nyc_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26256834 entries, 0 to 26256833
Data columns (total 41 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   Unique Key                      int64  
 1   Created Date                    object 
 2   Closed Date                     object 
 3   Agency                          object 
 4   Agency Name                     object 
 5   Complaint Type                  object 
 6   Descriptor                      object 
 7   Location Type                   object 
 8   Incident Zip                    object 
 9   Incident Address                object 
 10  Street Name                     object 
 11  Cross Street 1                  object 
 12  Cross Street 2                  object 
 13  Intersection Street 1           object 
 14  Intersection Street 2           object 
 15  Address Type                    object 
 16  City                            object 
 17  Landmark                 

In [7]:
nyc_311.shape

(26256834, 41)

## Part 2 - Columns 
________________________
The rest of the sections in this workbook deal with cleaning, formatting and filtering the dataset to the proper specifications for use in the analysis. The goal is to make sure the complaints are valid complaints and not errors or duplicates by verifying key columns. 
This section specifically deals with: 
1. Dropping unwanted columns - since the dataset is so large unneccessary, redundant or empty columns will be dropped first to avoid spending time and computational power on information that will be removed anyway. 
2. Formatting the column data types - the next step involves formatting the remaining columns into their correct datatypes so that they can be more easily worked with.

### Dropping Columns 
_________________________________________________________________________
This section provides a brief description of the columns being removed and the reasoning behind their removal from the dataset.

This list displays all of the columns in the dataset alongside the number of null values within each column.

In [8]:
# Displaying all of the columns and their missing values
nyc_311.isna().sum()

Unique Key                               0
Created Date                             0
Closed Date                         726090
Agency                                   0
Agency Name                              0
Complaint Type                           0
Descriptor                          395623
Location Type                      6292076
Incident Zip                       1397334
Incident Address                   4389721
Street Name                        4391260
Cross Street 1                     8287165
Cross Street 2                     8406104
Intersection Street 1             19490857
Intersection Street 2             19497258
Address Type                       4052072
City                               1630394
Landmark                          23178569
Facility Type                     20754865
Status                                   0
Due Date                          17588987
Resolution Description              544603
Resolution Action Updated Date      399433
Community B

#### Empty Columns 
These columns have such high numbers of missing values that the columns are rendered useless and will be dropped as a group. 

In [9]:
# Dropping all of these columns which have nothing but null values
nyc_top = nyc_311.drop(columns = ["Vehicle Type", 
                                  "Taxi Company Borough", 
                                  "Taxi Pick Up Location", 
                                  "Bridge Highway Name", 
                                  "Bridge Highway Direction", 
                                  "Road Ramp", 
                                  "Bridge Highway Segment"
                                  ])

#### Redundant Columns
These columns contain information that is found elsewhere within the table and are considered to be redundant. Each will be listed with a brief description of the column and the reason for removal.

- "Agency Name" - this column is already encoded in the "Agency" column, if the full name of the agency is needed it can be found easily. Because of this redundancy the column will be dropped.
All location data except for the "Latitude" and "Longitude columns will be dropped to reduce conflict between the variables as most of the other data columns are redundant.
- "Landmark" - this column indicates if there is a notable landmark associated with the location of the service request, it is redundant since latitude and longitude data are available. And because the sheer number of landmarks within the dataset puts this attribute outside the scope of this analysis.
- "X Coordinate (State Plane)" & "Y Coordinate (State Plane)" - these columns show the grid locations of each service request on the state unified plane. This plane is designed to help the government entities within the State of New York better coordinate location information in a single unified way. However for the purposes of this analsis this information is redundant and will be dropped. Source: https://gis.ny.gov/coordinationprogram/workgroups/wg_1/related/standards/datum.htm
- "Intersection Street 1" & "Intersection Street 2" - these columns display the nearest intersections on either side of the service request location, they are used for referencing blocks within each street. This information adds unneccessary complexity for the purposes of this analysis, the location of the requests will be encoded through the "Latitude" and "Longitude" columns instead.
- "Location" - this column is just a combination of the "Latitude" & "Longitude" columns and is redundant, so it will be dropped. 
- "Complaint Type" - the complaint type is encoded by the "Descriptor" column and is therefore redundant.
- "Incident Zip" - indicates the zip code of the complaint. 
- "Incident Address" - indicates the house/building/apartment number referenced in the complaint.
- "Street Name" - name of the street for the complaint.
- "Cross Street 1" & "Cross Street 2" - these columns show the nearest cross streets to the complaint location.
- "Location Type" - this column indicates the type of location the service request was made regarding, such as subway, sidewalk, street etc.
- "Address Type" - references the type of address for the complaint, whether it is a block address or a house/building address.
- "City" - shows the city within NYC that the complaint is from.
- "Borough" - shows the borough within NYC that the complaint is from.
- "Park Facility Name" - name of the park referenced in the complaint, similar to "Landmark".
- "Park Borough" - shows the borough within NYC that the park referenced in the complaint is from.

Dropping the columns.

In [10]:
# Dropping the redundant columns
nyc_top = nyc_top.drop(columns = ["Agency Name",  # Already encoded in the Agency Field, the full names can be found by the acronym
                                  "Landmark",  # Name of landmark if applicable - Vast number of NaN values and largely unneeded
                                  "X Coordinate (State Plane)", # NY State Grid Coordinate- Redundant
                                  "Y Coordinate (State Plane)", # NY State Grid Coordinate- Redundant
                                  "Intersection Street 1", # Similiar to Cross Street but has more NaN values
                                  "Intersection Street 2", # Similiar to Cross Street but has more NaN values
                                  "Location", # Combination of latitude and longitude that is redundant
                                  "Complaint Type", # The category of the complaint, already encoded in "Descriptor"
                                  "Incident Zip", # Zip code of the complaint, redundant
                                  "Incident Address", # House/building/apartment number of complaint
                                  "Street Name", # Name of the street for the complaint
                                  "Cross Street 1", # Cross street nearest complaint address
                                  "Cross Street 2", # Second cross street nearest complaint address
                                  "Location Type", # Type of location referenced in complaint
                                  "Address Type", # The type of address provided for the complaint
                                  "City", # City of the complaint address
                                  "Borough", # Borough of the complaint address
                                  "Park Facility Name", # Name of the park referenced in the complaint
                                  "Park Borough" # Borough of the park referenced in the complaint
                                 ])

#### Unneeded Columns
These columns contain information that is either irrelevant or too granular for use in this analysis.

- "Community Board" - this is similar to a neighborhood council that has citizen involvement in the administration of government services. This data is excessive for the analysis being performed and the effects will most likely be captured within the location data as the boards are assigned to geographic areas.
- "Facility Type" - this column references the type of NYC government building that is subject to a complaint, but only if the complaint is about a government service or building. Irrelevant information for this analysis.
- "BBL" - indicates the parcel id for the building referenced in the complaint, used by certain departments such as city planning to more accurately define and accesss records for complaint buildings.
- "Due Date" - this column shows the date by which the NYC governmental agency responsible for the complaint should respond to the complaint. For this analysis the due date is irrelevant, although this data may be accessed as part of additional analysis to compare the predictions made by the modeling to the standard expected by the city.
- "Resolution Description" - this column gives the action taken by the responding agency to rectify the complaint. Although this column could be useful for determining the length and quality of response it is post hoc information. Meaning that it is given after the event has occured and cannot be used as a predictor variable.
- "Resolution Action Updated Date" - this column gives the timestamp of the last update to the resolution given by the responsible agency. This column is unnecessary as the metric of concern is the time to completion for each complaint not the last update.

Dropping the columns.

In [11]:
# Dropping the unneeded columns
nyc_top = nyc_top.drop(columns = ["Community Board", # Citizen Board Jurisdiction
                                  "Facility Type", # Type of NYC government building referenced in complaint
                                  "BBL", # Land Parcel ID
                                  "Due Date", # Date by which the department responsible for the complaint should respond to the complaint
                                  "Resolution Description", # Description of action taken by the responding agency
                                  "Resolution Action Updated Date" # When the resolution action was last updated 
                                 ])

The old index column for the complaints was carried over and is now irrelevant so it will be dropped.

### Top Ten Descriptors 
_____________________________

The "Descriptor" column in this data set contains descriptions of complaints submitted to the 311 system. These descriptions are more granular than complaint types and are better categorization method for complaints than "Complaint Type"s which are too broad to be useful. More analysis will be performed on these top ten descriptors, to facilitate this the descriptors will be updated to more informative names.

#### Getting the top ten descriptors

In [12]:
# Getting the number of complaints by descriptor
com_descriptor_num = nyc_top["Descriptor"].value_counts()
# Sorting by number of complaints
com_descriptor_num_sorted = com_descriptor_num.sort_values(ascending = False)
# Getting the top ten descriptors
top_ten_descriptors = com_descriptor_num_sorted[:10]

Creating read-outs for each of the top ten descriptors with the number of complaints and the break down of descriptors by complaint type.

In [13]:
# Initializing the empty descriptor list
descriptor_list = []

# Initiating a for llop over the top ten descriptors for compalints
for counter, descriptor in enumerate(top_ten_descriptors):
    
    # Assigning the index of the descriptor to a variable
    index = top_ten_descriptors.index[counter]
    
    # Assigning the index of the complaint type to a variable
    value = top_ten_descriptors[counter]
    
    # Creates a readout of the complaint types for the descriptor in question
    print(f"The complaints with the {index} descriptor fall under these complaint types:")
    complaint_type = nyc_311["Complaint Type"].loc[nyc_311["Descriptor"] == index].value_counts()
    print(complaint_type)
    
    # Creates a readout showing the descriptor and the total number of complaints with that descriptor
    descriptor_var = nyc_311.loc[nyc_311["Descriptor"] == index]
    print(f"The total number of complaints with the {index} descriptor is {len(descriptor_var)}")
    print("\n")
    
    # Appends the descriptor to the descriptor list
    descriptor_list.append(index)

The complaints with the Loud Music/Party descriptor fall under these complaint types:
Noise - Residential         1631891
Noise - Street/Sidewalk      604129
Noise - Commercial           317268
Noise - Park                  38792
Noise - House of Worship       7729
Name: Complaint Type, dtype: int64
The total number of complaints with the Loud Music/Party descriptor is 2599809


The complaints with the ENTIRE BUILDING descriptor fall under these complaint types:
HEAT/HOT WATER    981577
Name: Complaint Type, dtype: int64
The total number of complaints with the ENTIRE BUILDING descriptor is 981577


The complaints with the HEAT descriptor fall under these complaint types:
HEATING    868960
Name: Complaint Type, dtype: int64
The total number of complaints with the HEAT descriptor is 868960


The complaints with the No Access descriptor fall under these complaint types:
Blocked Driveway    837875
Name: Complaint Type, dtype: int64
The total number of complaints with the No Access descript

The descriptors above will be the primary method of categorizing complaints used in this analysis, for the sake of readability the descriptor columns will be amended to give more context to the descriptor. For example "ENTIRE BUILDING" is too vauge to be useful, it will become "ENTIRE BUILDING - HEAT/HOT WATER OUT" which better conveys the needed information. This must be done manually as there are multiple complaint types for several descriptors.

#### 1. "Loud Music/Party" -> "LOUD MUSIC/PARTY - NOISE"
This descriptor is used for any loud music/party sounds, its classified as a general noise complaint that has a complaint type based on where the party/loud music is reported.
However it will just be labeled as "- NOISE" for simplicity as that conveys enough information.

In [14]:
# Creating a variable for the descriptor mask
describe = nyc_top["Descriptor"] == "Loud Music/Party"

#Changing all of the values for the descriptor to the annotated version
nyc_top.loc[describe, "Descriptor"] = "LOUD MUSIC/PARTY - NOISE"

#### 2. "ENTIRE BUILDING" -> "ENTIRE BUILDING - HEAT/HOT WATER OUT"
This descriptor indicates that the most or all of a buildings heating or hot water has gone out. There are several other heating related descriptors in the top ten that deal with outages that are either unknown or specific to a single apartment. 


In [15]:
# Creating a variable for the descriptor mask
describe = nyc_top["Descriptor"] == "ENTIRE BUILDING"

#Changing all of the values for the descriptor to the annotated version
nyc_top.loc[describe, "Descriptor"] = "ENTIRE BUILDING - HEAT/HOT WATER OUT"

#### 3. "HEAT" -> "HEAT OUT - GENERAL"
This descriptor indicates a heat outage of an unknown size. It is given the designation "- GENERAL" as not much more is known about the scale of outages for this descriptor.

In [16]:
# Creating a variable for the descriptor mask
describe = nyc_top["Descriptor"] == "HEAT"

#Changing all of the values for the descriptor to the annotated version
nyc_top.loc[describe, "Descriptor"] = "HEAT OUT - GENERAL"

#### 4. "No Access" -> "NO ACCESS - BLOCKED DRIVEWAY"
This descriptor indicates a blocked driveway complaint. Because the only complaint category for this descriptor is for a blocked driveway it has been labeled "- BLOCKED DRIVEWAY" to give enough context to interpret the descriptor.

In [17]:
# Creating a variable for the descriptor mask
describe = nyc_top["Descriptor"] == "No Access"

#Changing all of the values for the descriptor to the annotated version
nyc_top.loc[describe, "Descriptor"] = "NO ACCESS - BLOCKED DRIVEWAY"

#### 5. "Street Light Out" -> "STREET LIGHT OUT"
This descriptor is self explantory and will only be changed into all capital letters for the sake of consistency.

In [18]:
# Creating a variable for the descriptor mask
describe = nyc_top["Descriptor"] == "Street Light Out"

#Changing all of the values for the descriptor to the annotated version
nyc_top.loc[describe, "Descriptor"] = "STREET LIGHT OUT"

#### 6. "Banging/Pounding" -> "BANGING/POUNDING - NOISE"
This descriptor details a specific banging/pounding noise complaint, "- NOISE" will be added for clarity.

In [19]:
# Creating a variable for the descriptor mask
describe = nyc_top["Descriptor"] == "Banging/Pounding"

#Changing all of the values for the descriptor to the annotated version
nyc_top.loc[describe, "Descriptor"] = "BANGING/POUNDING - NOISE"

#### 7. "Pothole" -> "POTHOLE"
The descriptor is fairly self explanatory, the descriptor will be formatted in all capital letters for consistency.

In [20]:
# Creating a variable for the descriptor mask
describe = nyc_top["Descriptor"] == "Pothole"

#Changing all of the values for the descriptor to the annotated version
nyc_top.loc[describe, "Descriptor"] = "POTHOLE"

#### 8. "APARTMENT ONLY" -> "APARTMENT ONLY - HEAT/HOT WATER OUT"
This descriptor is a continuation of the "Heat/Hot Water Out" complaint type and indicates an apartment specific outage.

In [21]:
# Creating a variable for the descriptor mask
describe = nyc_top["Descriptor"] == "APARTMENT ONLY"

#Changing all of the values for the descriptor to the annotated version
nyc_top.loc[describe, "Descriptor"] = "APARTMENT ONLY - HEAT/HOT WATER OUT"

#### 9. "Loud Talking" -> "LOUD TALKING - NOISE"
This descriptor is a continuation of the "- Noise" complaint type and will be marked with " - NOISE" to indicate a noise complaint.

In [22]:
# Creating a variable for the descriptor mask
describe = nyc_top["Descriptor"] == "Loud Talking"

#Changing all of the values for the descriptor to the annotated version
nyc_top.loc[describe, "Descriptor"] = "LOUD TALKING - NOISE"

#### 10. "CEILING" -> "CEILING - PAINT/PLASTER REPAIR"
This descriptor indicates a complaint filed with the city alledging that a repair to fix a ceiling with paint or plaster is not being performed by the owners or co-op board of an apartment building. 
By law these repairs must be made to ensure livability, the descriptor will be annotated with "- PAINT/PLASTER REPAIR" to give context.
Source: https://portal.311.nyc.gov/article/?kanumber=KA-01074

In [23]:
# Creating a variable for the descriptor mask
describe = nyc_top["Descriptor"] == "CEILING"

#Changing all of the values for the descriptor to the annotated version
nyc_top.loc[describe, "Descriptor"] = "CEILING - PAINT/PLASTER REPAIR"

#### Checking that the descriptor renaming worked by recompiling the top ten descriptors.

In [24]:
# Getting the number of complaints by descriptor
com_descriptor_num = nyc_top["Descriptor"].value_counts()
# Sorting by number of complaints
com_descriptor_num_sorted = com_descriptor_num.sort_values(ascending = False)
# Getting the top ten descriptors
top_ten_descriptors = com_descriptor_num_sorted[:10]
# Displaying the top ten descriptors
top_ten_descriptors

LOUD MUSIC/PARTY - NOISE                2599809
ENTIRE BUILDING - HEAT/HOT WATER OUT     981577
HEAT OUT - GENERAL                       868960
NO ACCESS - BLOCKED DRIVEWAY             837875
STREET LIGHT OUT                         751210
BANGING/POUNDING - NOISE                 661458
POTHOLE                                  646377
APARTMENT ONLY - HEAT/HOT WATER OUT      525424
LOUD TALKING - NOISE                     386836
CEILING - PAINT/PLASTER REPAIR           368831
Name: Descriptor, dtype: int64

In [25]:
# Summing the total number of compaints
top_ten_sum = top_ten_descriptors.sum()
print(f"The total number of complaints for the top ten descriptors is {top_ten_sum} out of {len(nyc_top)} total complaints.")

The total number of complaints for the top ten descriptors is 8628357 out of 26256834 total complaints.


### Formatting the column data types
_______________________________________________
One of the columns within the dataset has a sub-optimal datatype for use in the cleaning process. The change in datatype and the reasoning behind it is explained below.

In [26]:
# Looking at the datatypes for each of the columns
nyc_top.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26256834 entries, 0 to 26256833
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Unique Key              int64  
 1   Created Date            object 
 2   Closed Date             object 
 3   Agency                  object 
 4   Descriptor              object 
 5   Status                  object 
 6   Open Data Channel Type  object 
 7   Latitude                float64
 8   Longitude               float64
dtypes: float64(2), int64(1), object(6)
memory usage: 1.8+ GB


Most of the columns are in acceptable formats, the vast majority are classed as objects because they contain strings.

The "Created Date" column needs to be converted into 'datetime64[ns]' format, this will allow for more advanced functions to be performed on the column and making working with the time data easier. 

In [27]:
start = datetime.now()
# Converting the "Created Date"
nyc_top["Created Date"] = pd.to_datetime(nyc_top["Created Date"])
finish = datetime.now()
duration = finish - start
print(duration)

0:31:09.053874


In [28]:
# Checking for successful conversion
nyc_top.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26256834 entries, 0 to 26256833
Data columns (total 9 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Unique Key              int64         
 1   Created Date            datetime64[ns]
 2   Closed Date             object        
 3   Agency                  object        
 4   Descriptor              object        
 5   Status                  object        
 6   Open Data Channel Type  object        
 7   Latitude                float64       
 8   Longitude               float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 1.8+ GB


Now that the data types have been successfuly changed a backup copy of the data will be created. This is just for the convenience of the viewer so that the computationally expensive process of re-formatting the columns isn't necessary if the data needs to be reset.

In [29]:
# Creating a backup copy to avoid reformatting columns if an error occurs
nyc_top_backup = nyc_top
nyc_top_backup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26256834 entries, 0 to 26256833
Data columns (total 9 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Unique Key              int64         
 1   Created Date            datetime64[ns]
 2   Closed Date             object        
 3   Agency                  object        
 4   Descriptor              object        
 5   Status                  object        
 6   Open Data Channel Type  object        
 7   Latitude                float64       
 8   Longitude               float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 1.8+ GB


In [30]:
# Code to reset nyc_top to backup state
nyc_top = nyc_top_backup

## Part 3 - Rows 
___________________________
The next step in cleaning this dataset is to address inconsistencies within the data. Primarily null or duplicate values. Context for each set of missing data and the actions taken to address the issue are provided below in addition to the reasoning for each decision. 

### Dealing with null values
__________________________________________


In [31]:
# Displaying all of the null value counts for the dataframe
nyc_top.isna().sum()

Unique Key                      0
Created Date                    0
Closed Date                726090
Agency                          0
Descriptor                 395623
Status                          0
Open Data Channel Type          0
Latitude                  2141807
Longitude                 2141807
dtype: int64

#### Latitude and Longitude

In [32]:
# Number of rows in the dataset
rows = len(nyc_top)

# Creating masks for null values
lat_na = len(nyc_top[nyc_top["Latitude"].isna() == True])
lon_na = len(nyc_top[nyc_top["Longitude"].isna() == True])

# Printing the number of rows with null values for the column
print(f"The number of rows with NaN latitude values is {lat_na}.")
print(f"The number of rows with NaN longitude values is {lon_na}.")

# Displaying the number of rows in the dataset
print(f"The total number of rows in the dataset is {rows}.")

# Displaying the percentage of rows in the dataset with missing values
print(f"The rows with missing location values account for {round((lat_na/rows)*100,2)}% of the dataset.")

The number of rows with NaN latitude values is 2141807.
The number of rows with NaN longitude values is 2141807.
The total number of rows in the dataset is 26256834.
The rows with missing location values account for 8.16% of the dataset.


All of the rows that have missing "Latitude" values also have missing "Longitude" values, this suggests that the geolocation data usually obtained by the responding agency was corrupted, lost or never collected. Resulting in an address and associated information being provided without the geolocation data verification. 

In [33]:
# NOTE: nyc_311 was used to check the number of missing values per address 

# Calculating the number of missing "Latitude" values for each unique address within the data
counts_addresses = nyc_311["Incident Address"][nyc_311["Latitude"].isna()].value_counts() 

# Calculating the number of unique addresses in the dataset
unique_addresses = len(nyc_311["Incident Address"][nyc_311["Latitude"].isna()].unique())

# Calculating the percentage of missing "Latitude" and "Longitude" data for the address with the most missing data
most_missing = round((counts_addresses[0]/lat_na)*100, 2)

# Printing various statistics 
print("The five addresses with the most missing location data: \n", counts_addresses.head(5))
print("\n")
print("The five addresses with the least missing location data: \n", counts_addresses.tail(5))
print("\n")
print(f"There are {unique_addresses} unique addresses with missing Latitude and Longitude data.")
print(f"The address with the most missing values accounts for just {most_missing}% of the missing Latitude and Longitude data.")

The five addresses with the most missing location data: 
 BELT PKWY                5713
BKLYN QUEENS EXPY        5001
CENTRAL PARK             4711
GRAND CENTRAL PKWY       4557
FLUSHING MEADOWS PARK    4439
Name: Incident Address, dtype: int64


The five addresses with the least missing location data: 
 96 86 STREET           1
90-29 56 AVENUE        1
155 NEW DORP LANE      1
1325 EAST 29 STREET    1
145105 DIVISION AVE    1
Name: Incident Address, dtype: int64


There are 165763 unique addresses with missing Latitude and Longitude data.
The address with the most missing values accounts for just 0.27% of the missing Latitude and Longitude data.


Due to the scope of missing data as illustrated above the rows with missing "Latitude" and "Longitude" data will be dropped. It is impractical due to the time constraints on this analysis to attempt to rectify such a large number of missing values in such small increments. As noted above the rows with missing values account for only 4.58% of the dataset, and since the entire dataset is 8.62 million rows long the loss of under 400 thousands rows should not effect the viability of the analysis or the resulting models. 

In [34]:
# Filtering out all of the rows with missing "Latitude" data
nyc_top = nyc_top[nyc_top["Latitude"].isna() == False]

# Checking how many rows have missing "Latitude" values
missing_lat = len(nyc_top[nyc_top["Latitude"].isna() == True])

# Checking if there are any rows with undesired values
print(f"Rows with missing latitude values: {missing_lat}")

Rows with missing latitude values: 0


The filtering of missing "Latitude" data was successful, as expected all of the rows with missing longitude data were filtered out as well.

#### Closed Date

In [35]:
# Displaying all of the null value counts for the dataframe
nyc_top.isna().sum()

Unique Key                     0
Created Date                   0
Closed Date               684826
Agency                         0
Descriptor                320894
Status                         0
Open Data Channel Type         0
Latitude                       0
Longitude                      0
dtype: int64

In [36]:
# Number of rows in the dataset
len(nyc_top)

# Checking how many rows have missing "Closed Date" values
closed_na = len(nyc_top[nyc_top["Closed Date"].isna() == True])
print(f"The number of rows with missing Closed Dates is {closed_na}.")

# Calculating and displaying the percentage of rows being dropped
print(f"The rows that have missing Closed Dates for {round((closed_na/rows)*100,2)}% of the dataset.")

The number of rows with missing Closed Dates is 684826.
The rows that have missing Closed Dates for 2.61% of the dataset.


The "Closed Date" column shows the day on which the complaint was considered closed by the agency responsible for the complaint. For this analysis only those complaints that are closed will be considered. The majority of the missing values come from "Open" complaints but some arise from errors or misclassifications. The rows with missing "Closed Date"s will be dropped. Additional filtering will take place in a subsequent step to filter out all statuses other than closed complaints.

In [37]:
# Filtering out all of the rows with missing "Closed Date" data
nyc_top = nyc_top[nyc_top["Closed Date"].isna() == False]

# Checking how many rows have missing "Latitude" values
missing_closed = len(nyc_top[nyc_top["Closed Date"].isna() == True])

# Checking if there are any rows with undesired values
print(f"Rows with missing Closed Dates: {missing_closed}")

Rows with missing Closed Dates: 0


The drop was succesful. Only one more set of missing data values remains.

In [38]:
nyc_top.isna().sum()

Unique Key                     0
Created Date                   0
Closed Date                    0
Agency                         0
Descriptor                314438
Status                         0
Open Data Channel Type         0
Latitude                       0
Longitude                      0
dtype: int64

In [39]:
# Number of rows in the dataset
len(nyc_top)

# Checking how many rows have missing "Descriptor" values
descriptor_na = len(nyc_top[nyc_top["Descriptor"].isna() == True])
print(f"The number of rows with missing descriptors is {descriptor_na}.")

# Calculating and displaying the percentage of rows being dropped
print(f"The rows that have missing descriptors account for {round((descriptor_na/rows)*100,2)}% of the dataset.")

The number of rows with missing descriptors is 314438.
The rows that have missing descriptors account for 1.2% of the dataset.


The "Descriptor" columns give the subset of complaint type for the SR request. Only those SRs that have a valid "Descriptor" value will be considered to ensure only valid SRs are used. 

In [40]:
# Filtering out all of the rows with missing "Descriptor" data
nyc_top = nyc_top[nyc_top["Descriptor"].isna() == False]

# Checking how many rows have missing "Latitude" values
missing_descriptor = len(nyc_top[nyc_top["Descriptor"].isna() == True])

# Checking if there are any rows with undesired values
print(f"Rows with missing Closed Dates: {missing_descriptor}")

Rows with missing Closed Dates: 0


The drop was successful. The next step is searching for duplicates within the dataset.

### Checking for duplicates
_______________________________

In [41]:
nyc_top.duplicated().value_counts()

False    23115763
dtype: int64

There don't appear to be any duplicate values.

### Resetting the Index
__________________________

Now that all of the drops have been performed on the datasets the index will be reset to make the index values contiguous. 

In [42]:
# Resetting the index
nyc_top.reset_index(drop = True, inplace = True)

In [43]:
# Checking for successful completion
nyc_top.head(3)

,Unique Key,Created Date,Closed Date,Agency,Descriptor,Status,Open Data Channel Type,Latitude,Longitude
0,48330841,2020-12-05 22:00:30,12/05/2020 10:14:35 PM,NYPD,LOUD MUSIC/PARTY - NOISE,Closed,MOBILE,40.841317,-73.936608
1,48330856,2020-12-05 21:19:55,12/05/2020 10:49:08 PM,NYPD,LOUD MUSIC/PARTY - NOISE,Closed,MOBILE,40.833438,-73.945336
2,48330864,2020-12-06 00:38:54,12/06/2020 01:06:35 AM,NYPD,LOUD MUSIC/PARTY - NOISE,Closed,ONLINE,40.707605,-74.005720


The index column was successfully reset.

## Part 4 - Limiting Scope
___________________________________________________________________
This section deals with limiting the dataset to only those complaints that have both a "Closed" status and those complaints filed in 2010-2019. 

### Status
________________________


In [44]:
# Number of values for each possible complaint "Status"
nyc_top["Status"].value_counts()

Closed              22949858
Pending               130886
Assigned               17239
Open                   17117
In Progress              502
Started                  147
Closed - Testing           9
Unassigned                 5
Name: Status, dtype: int64

In [45]:
# Number of rows in the dataset
rows = len(nyc_top)

# Number of rows with statuses other than "Closed"
status = len(nyc_top[nyc_top["Status"] != "Closed"] )

# Displaying the number of rows with statuses other than "Closed"
print(f"The number of rows with statuses other than Closed is {status}.")

# Calculating and displaying the percentage of rows being dropped
print(f"The rows that have statuses other than Closed account for {round((status/rows)*100,2)}% of the dataset.")

The number of rows with statuses other than Closed is 165905.
The rows that have statuses other than Closed account for 0.72% of the dataset.


The "Status" column describes the stage of the complaint handling process each individual complaint is at. A complaint marked "Closed" has been responded to and any appropriate actions have been completed by the responsible agencies. For the purposes of this analysis only those complaints that are marked as "Closed" will be considered and all other complaints will be dropped.

In [46]:
# Filtering out all of the rows with missing "Latitude" data
nyc_top = nyc_top[(nyc_top["Status"] == "Closed")]

# Checking how many rows have missing "Latitude" values
unclosed = len(nyc_top[nyc_top["Status"] != "Closed"])

# Checking if there are any rows with undesired values
print(f"Rows with unclosed statuses: {unclosed}")

Rows with unclosed statuses: 0


The filtering for unclosed complaints was successful. Now that only closed complaints are present within the dataframe the "Status" column will be dropped as it is no longer necessary.

In [47]:
# Dropping the "Status" column
nyc_top.drop(labels = "Status", axis = 1, inplace = True)

### Timeframe
______________________________

In [48]:
# Displaying the number of complaints by year
nyc_top["Unique Key"].groupby(nyc_top["Created Date"].dt.year).agg("count")

Created Date
2010    1722919
2011    1569560
2012    1543714
2013    1577237
2014    1865478
2015    2021295
2016    2090550
2017    2211769
2018    2474355
2019    2160656
2020    2333861
2021    1378464
Name: Unique Key, dtype: int64

The original dataset being used for this analysis consisted of 311 complaints from NYC that were collected from 2010 until the time of this analysis in September 2021. Such a large timeframe makes for an unwieldy amount of data for the resources available to this project. In addition since the start of the COVID-19 pandemic in March of 2020 the collection capabilty and quality of data in 311 complaints has suffered drastically. This is due to city services being overwhelmed at in almost every possible way. Leading to a backlog of complaints and failures in data collection. To counter the issues of both size and data quality this analysis will focus only on complaints CLOSED from 2010 to December 2019 and the data will be further reduced in to timeseries format.

In [49]:
# Filtering out complaints from 2020 and 2021
nyc_top = nyc_top[nyc_top["Created Date"].dt.year <= 2019]

# Checking how many rows are from 2020 and 2021
years = len(nyc_top[nyc_top["Created Date"].dt.year >= 2020])

# Checking if there are any rows with undesired values
print(f"Number of complaints from 2020 and 2021: {years}")

Number of complaints from 2020 and 2021: 0


### Converting the complain dataframe to a .csv file
_____________________________
The filtering was successful, the data set is now ready for conversion to timeseries format. But the dataframe will first be converted into a .csv for use in EDA in its current form.

In [50]:
# Checking the dataframe shape
nyc_top.shape

(19237533, 8)

In [51]:
nyc_top.reset_index(drop = True, inplace = True)

In [52]:
# Displaying final information
nyc_top.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237533 entries, 0 to 19237532
Data columns (total 8 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   Unique Key              int64         
 1   Created Date            datetime64[ns]
 2   Closed Date             object        
 3   Agency                  object        
 4   Descriptor              object        
 5   Open Data Channel Type  object        
 6   Latitude                float64       
 7   Longitude               float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 1.1+ GB


In [53]:
start = datetime.now()
# Converting the dataframe into a .csv file
nyc_top.to_csv('311 Data/nyc_clean_complaint_311.csv')
finish = datetime.now()
duration = finish - start
print(duration)

0:01:27.978603


## Part 5 - Creating the timeseries dataframe
________________________________

### Creating the dataset

The process of taking the cleaned dataset and converting it into a timeseries format is quite simple. Timeseries data tracks only time as an independent variable (X), with the same dependent variables shown through time (y). In this use case there will be only the time index by day and the total number of SRs submitted that day. An aggregation function is used to group all of the SRs together by date and calculate the per day values. 

In [54]:
# Grouping the SRs by day and counting the per day values
totals = nyc_top["Unique Key"].groupby(nyc_top["Created Date"].dt.date).count()

Now that the values have been calculated a dataframe is made and the columns are renamed to reflect their contents.

In [55]:
# Creating a dataframe from the totals
time_df = pd.DataFrame(totals)

In [56]:
# Renaming the columns
time_df.rename(columns = {"Unique Key": "Total SRs"}, inplace = True)

Now we can see the first few rows and information on the dataset.

In [57]:
# Displaying basic info on the dataset
time_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3652 entries, 2010-01-01 to 2019-12-31
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Total SRs  3652 non-null   int64
dtypes: int64(1)
memory usage: 57.1+ KB


In [58]:
# Displaying the first five rows of the dataset
time_df.head(5)

,Total SRs
Created Date,
2010-01-01,2782
2010-01-02,3692
2010-01-03,5426
2010-01-04,7977
2010-01-05,7119


### Cleaning the new dataset

Now that the data has been successfully gathered and formatted it can be checked for inconsistencies or other problems. Given the synthesized and pre-cleaned nature of the dataset there are likely few issues, but the checks will take place nonetheless.

The first step in cleaning a timeseries dataset is calculating the date range for the dataset.

In [59]:
# Finding and printing the first day of the dataset
first_day = time_df.index.min()
print(first_day)

# Finding and printing the last day of the dataset
last_day = time_df.index.max()
print(last_day)

# Finding and printing the date range for the dataset
full_range = pd.date_range(start=first_day, end=last_day, freq="D")
print(full_range)

2010-01-01
2019-12-31
DatetimeIndex(['2010-01-01', '2010-01-02', '2010-01-03', '2010-01-04',
               '2010-01-05', '2010-01-06', '2010-01-07', '2010-01-08',
               '2010-01-09', '2010-01-10',
               ...
               '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25',
               '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', length=3652, freq='D')


Now the date range of the dataset can be checked against the index of the dataframe, these values should have no differences, meaning that every day within the date range is present in the index and date range.

In [60]:
full_range.difference(time_df.index)

DatetimeIndex([], dtype='datetime64[ns]', freq='D')

The check was successful and found no evidence of missing days. The only remaining step for the cleaning is checking for null values which is done below.

In [61]:
time_df.isna().sum()

Total SRs    0
dtype: int64

There were no null values. 

## Part 6 - Conversion to CSV
________________________________

Now that all of the necessary cleaning and formatting for this dataset is complete the dataset will be turned into a .csv file for use in other notebooks. 

### Final dimensions check
_____________________________
Now the dimensions and first few rows will be checked to verify everything is as desired

In [62]:
# Displaying the first five rows of the new dataframe
time_df.head(5)

,Total SRs
Created Date,
2010-01-01,2782
2010-01-02,3692
2010-01-03,5426
2010-01-04,7977
2010-01-05,7119


In [63]:
# Displaying the columns and datatypes of the final dataframe
time_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3652 entries, 2010-01-01 to 2019-12-31
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Total SRs  3652 non-null   int64
dtypes: int64(1)
memory usage: 57.1+ KB


In [64]:
# Checking the shape of the final dataframe
time_df.shape

(3652, 1)

The final step in this notebook is converting the final dataframe into a .csv file for use in the upcoming EDA and modeling.

In [65]:
start = datetime.now()
# Converting the dataframe into a .csv file
time_df.to_csv('311 Data/nyc_timeseries_311.csv')
finish = datetime.now()
duration = finish - start
print(duration)

0:00:00.006004


In [66]:
# Calculating and displaying the notebook runtime
e_end = datetime.now()
notebook_runtime = e_end - e_start
print(notebook_runtime)

0:37:19.954688
